# Shap Values

La interpretabilidad de los modelos es un "tema caliente", en muchas ocasiones no podremos entregar un modelo que sea una caja negra, donde no podamos explicar que está pasando realmente. 

En ciertas aréas es particularmente dificil adoptar modelos que sean cajas negras (medicina, banca, ... ), por lo tanto, cuanto mejor sea la interpretabilidad del modelo, mejor adopción tendremos.

Para conseguir esta interpretabilidad de los modelos, podemos utilizar distintas herramientas: 

  - **Shap Values**
  - Lime
  - InterpretML
  - ELI5
  
A lo largo de este tema nos vamos a centrar en la primera, los SHAP Values.

![](img/shap.svg)

SHAP significa **SHapley Additive exPlanations**, de este modo, para entender que son los SHAP values tenemos que saver que es un Shapley value. 

Un SHapley value es la media de las contribuciones marginales de cada elemento en las diferentes permutaciones de estos [Definicion matemática](https://math.stackexchange.com/questions/111580/shapley-value-formula#:~:text=I%20understand%20Shapley%20value%20in,%E2%88%92v(s)).

Una vez conocemos que es un SHapley value, vamos a ver que es SHAP:

[Lundberg & Lee (2016)](https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf) propusieron el SHAP value como una aproximación unida para explicar los resultados de cualquier modelo de machine learning, otorgandonos los siguientes beneficios: 

1. Interpretabilidad global. Los valores agregados de SHAP values nos indica cuanto contribuye cada predictor.
1. Interpretabilidad local. Cada observacion tiene su conjunto de SHAP values, lo que nos da transparencia.
1. Posibilidad de calcular SHAP para cualquier modelo basado en arboles.

Dicho esto, vamos a ver como visualizar la explicabilidad de los modelos con SHAP en python.

## 1 - importamos las librerías de base

In [ ]:
## common libraries for Data Wrangling and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## sklearn: train test split, RandomForestRegressor, y mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## import shap
import shap

## Carga los datos en un df

In [ ]:
df = pd.read_csv('data/winequality-red.csv', sep=';')

In [ ]:
df.sample(10)

## Separar los datos en train + test (80% - 20%)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=["quality"]),
                                                    df['quality'],
                                                    test_size = ##,
                                                    random_state=0)

## Entrerna un RandomForestRegressor como modelo para predecir la calidad del vino

In [ ]:
model = RandomForestRegressor(n_estimators=100,
                              max_depth=4,
                              random_state=0)

## fit model with train data
model.##

## Evalua in-sample y out-of-sample para comprobar que no estamos cometiendo overfitting

In [ ]:
y_pred_train = model.predict(##)
y_pred_test = model.predict(##)

mae_in_sample = np.sqrt(mean_squared_error(y_true=Y_train, y_pred=y_pred_train))
mae_out_of_sample = np.sqrt(mean_squared_error(y_true=Y_test, y_pred=y_pred_test))

In [ ]:
print(f'In sample error: {mae_in_sample}')
print(f'Out sample error: {mae_out_of_sample}')

## Visualiza la importancia de las características que devuelve el `Random Forest`

In [ ]:
importances = model.##
features = X_train.##

feat_importance = pd.DataFrame({'feature': features, 'importance': importances})

In [ ]:
feat_importance.set_index('feature')['importance'].sort_values().plot(kind='barh')

# Inspección de modelo usando SHAP

Ahora vamos a ver lo que podemos obtener con SHAP, y si en primera instancia los valores son similares:

In [ ]:
explainer = shap.Explainer(model=##)

In [ ]:
shap_values = explainer(##)

## Feature importance

In [ ]:
shap.plots.bar(##)

Vemos algún cambio menor, pero en lineas generales los valores son similares, pero de momento, no estamos ganando nada con respecto a la importancia de las variables que nos otorga RandomForest.

Una cosa que podemos hacer con SHAP es obtener el signo del impacto, es decir, saber si las variables tienen un impacto positivo o negativo en el resultado final, vamos a verlo: 

## Summary plot

In [ ]:
shap.summary_plot(shap_values=##)

En este grafico podemos ver la siguiente información: 

- El eje vertical muestra la importancia de la variable 
- El eje horizontal muestra el efecto de cada punto para la predicción
- El color muestra si el valor de la variable fue alto o bajo

De este modo podemos empezar a entender nuestro modelo, sabiendo por ejemplo que un buen vino tiene: 

  - Alto contenido en alcohol
  - Alto contenido en sulfatos
  - Baja volatilidad con la acided
  - Bajo ph
  - ...

Pero todavía podemos ir mas allá y entender los impactos directos de cada una de las features a una predicción..

In [ ]:
shap.plots.force(shap_values[##])  ## el indice es el numero de linea del ejemplo del dataset